In [4]:
#So far the problem now is in the ticker code down on line 124. I'm assuming that's as far as the code is running, so possibly
#more problems after the line, but it's ok

from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi_a
import os
import urllib
import krakenex

load_dotenv('Alpaca.env')
kraken_key = os.environ.get('KRAKEN_PUBLIC_KEY')
kraken_secret_key = os.environ.get('KRAKEN_SECRET_KEY')
type(kraken_key)




from datetime import datetime
from dateutil.relativedelta import relativedelta

### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")

def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }


'''
def get_btcprice():
    """
    Retrieves the current price of bitcoin in US Dollars from the alternative.me Crypto API.
    """
    bitcoin_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"BTC"+"USD"
    response = requests.get(bitcoin_api_url)
    response_json = response.json()
    price_usd = response_json["result"]['XXBTZUSD']['a'][0]
    return price_usd

def get_ethprice():
    
    Ethereum_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"ETH"+"USD"
    response = requests.get(Ethereum_api_url)
    response_json = response.json()
    price_USDxETH = response_json["result"]['XETHZUSD']['a'][0]
    return price_USDxETH
#ethereum, bitcoin, ripple, chainlink, ada, litecoin, filecoin, dot
def get_rippleprice():
'''
    
#probably will be erased
def get_investment_help(helping_hand, ticker):
    """
    Returns an initial investment advice based on the choice of user.
    """
    helping_hands = {
        "ticker": "response",
        "fetch": "100% f"{ticker}",
        "buy": "100% f"{ticker}",
        "sell": "100% f"{ticker}",
        "stake": "100% f"{ticker}",
     
        
    } #do one function for buying, one for fetch, one for ...etc... define a function for the list of tickers

    return helping_hands[helping_hand()]

### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]

def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }

def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response

def convert_usd(intent_request):
    """
    Performs dialog management and fulfillment for recommending a portfolio.
    """

    # Gets slots' values
    birthday = get_slots(intent_request)["birthday"]
    usd_amount = get_slots(intent_request)["usdAmount"]

    # Gets the invocation source, for Lex dialogs "DialogCodeHook" is expected.
    source = intent_request["invocationSource"]  #

    if source == "DialogCodeHook":
        # This code performs basic validation on the supplied input slots.

        # Gets all the slots
        slots = get_slots(intent_request)

        # Validates user's input using the validate_data function
        validation_result = validate_data(birthday, usd_amount, intent_request)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attributes
        output_session_attributes = intent_request["sessionAttributes"]

        # Once all slots are valid, a delegate dialog is returned to Lex to choose the next course of action.
        return delegate(output_session_attributes, get_slots(intent_request))

    # Get the current price of BTC in USD and make the conversion from USD to BTC.
    btc_value = parse_float(usd_amount) / get_btcprice()
    btc_value = round(btc_value, 2)

    # Return a message with conversion's result.
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Thank you for your information;
            you can get {} Bitcoins for your {} US Dollars.
            """.format(
                btc_value, usd_amount
            ),
        },
    )


### Intents Handlers ###
def recommend_portfolio(intent_request):
    """
    Performs dialog management and fulfillment for recommending a portfolio.
    """

    first_name = get_slots(intent_request)["firstName"]
    age = get_slots(intent_request)["age"]
    investment_amount = get_slots(intent_request)["investmentAmount"]
    risk_level = get_slots(intent_request)["riskLevel"]
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        slots = get_slots(intent_request)

        validation_result = validate_data(age, investment_amount, intent_request)
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attibutes
        output_session_attributes = intent_request["sessionAttributes"]

        return delegate(output_session_attributes, get_slots(intent_request))

    # Get the initial investment recommendation
    initial_recommendation = get_investment_recommendation(risk_level)

    # Return a message with the initial recommendation based on the risk level.
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """{} thank you for your information;
            based on the risk level you defined, my recommendation is to choose an investment portfolio with {}
            """.format(
                first_name, initial_recommendation
            ),
        },
    )



# --------- User Settings ---------
API_KEY = "KRAKEN_KEY"
API_SECRET_KEY = "KRAKEN_SECRET_KEY"

# ---------------------------------
import json
def get_conditions():
    ticker = "ETH"
    api_conditions_url = "https://api.kraken.com/0/public/Ticker?pair="+ticker+"USD"
    
    response = urllib.request.urlopen(api_conditions_url)
   
    json_conditions = response.read()
    
    #f.close()
    return json.loads(json_conditions)
#Put the code in here
#import json
#ticker = "ETH"
#api_conditions_url = "https://api.kraken.com/0/public/Ticker?pair="+ticker+"USD"
#response = urllib.request.urlopen(api_conditions_url)
#data = json.load(response)
'''
conditions = get_conditions()
#coins = {'ETH':'https://www.kraken.com/prices/eth-ethereum-price-chart/usdc-usd-coin?interval=1m'}

if (conditions != False):
    
    ticker = "ETH"
    payload = urllib.parse.urlencode({f'{ticker} USD':conditions["ticker"]['USD']})
    
    
    payload1 = payload.encode('ascii')
    payload1 = payload1.request.Request(payload1, payload1)
    url = 'https://api.kraken.com/0/public/Ticker?pair=ETH' + '&apiSecretKey=' + API_SECRET_KEY + '&apiKey=' + API_KEY
    #url = 'https://www.kraken.com/api/events?apiSecretKey=' + API_SECREET_KEY + '&apiKey=' + API_SECRET_KEY
    urllib.request.urlopen(url,payload)
    exit()
    
'''

### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "RecommendPortfolio":
        return recommend_portfolio(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)




SyntaxError: invalid syntax (<ipython-input-4-bed38470635d>, line 63)

In [17]:
kraken = krakenex.API(key ='KRAKEN_PUBLIC_KEY', secret_key='KRAKEN_PUBLIC_KEY')

TypeError: __init__() got an unexpected keyword argument 'secret_key'

In [6]:
import requests
response=requests.get('https://api.kraken.com/0/public/Ticker?pair=xbtusd')

In [7]:
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


def get_btcprice():
    """
    Retrieves the current price of bitcoin in US Dollars from the alternative.me Crypto API.
    """
    bitcoin_api_url = "https://api.kraken.com/0/public/Ticker?pair=xbtusd"
    response = requests.get(bitcoin_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd



In [8]:
def parse_float(n):
    """
    Securely converts a non-numeric value to float.
    """
    try:
        return float(n)
    except ValueError:
        return float("nan")


def get_coin_price(coin):
    """
    Retrieves the current price of bitcoin in US Dollars from the kraken.com Crypto API.
    """
    bitcoin_api_url = "https://api.kraken.com/0/public/Ticker?pair="+coin+"USD"
    response = requests.get(bitcoin_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd

get_coin_price(ETH)

NameError: name 'ETH' is not defined

In [9]:
def get_coin_price(coin):
    """
    Retrieves the current price of bitcoin in US Dollars from the alternative.me Crypto API.
    """
    bitcoin_api_url = "https://api.alternative.me/v2/ticker/{coin}/?convert=USD"
    response = requests.get(bitcoin_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd

In [10]:
from dotenv import load_dotenv
import requests
import os
import urllib
import krakenex

load_dotenv('Alpaca.env')
kraken_key = os.environ.get('KRAKEN_PUBLIC_KEY')
kraken_secret_key = os.environ.get('KRAKEN_SECRET_KEY')
type(kraken_key)



def parse_float(n):
    """
    Securely converts a non-numeric value to float.
    """
    try:
        return float(n)
    except ValueError:
        return float("nan")


def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }


def get_coin_price(coin):
    """
    Retrieves the current price of bitcoin in US Dollars from the kraken.com Crypto API.
    """
    Ethereum_api_url = "https://api.kraken.com/0/public/Ticker?pair="+coin+"USD"
    response = requests.get(Ethereum_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd

get_coin_price("ETH")

KeyError: 'data'

In [16]:

Ethereum_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"ETH"+"USD"
response = requests.get(Ethereum_api_url)
response_json = response.json()
print(response_json["result"]['XETHZUSD']['a'][0])
'''
price_usd = parse_float(response_json["result"]["XETHZUSD"]['a'][0])
print(price_usd)
'''

2122.01000


'\nprice_usd = parse_float(response_json["result"]["XETHZUSD"][\'a\'][0])\nprint(price_usd)\n'

In [15]:
Ethereum_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"BTC"+"USD"
response = requests.get(Ethereum_api_url)
response_json = response.json()
print(response_json["result"])

{'XXBTZUSD': {'a': ['59333.10000', '1', '1.000'], 'b': ['59333.00000', '1', '1.000'], 'c': ['59333.10000', '0.37000000'], 'v': ['4835.80833716', '5295.31134127'], 'p': ['60032.55901', '59870.92420'], 't': [58385, 64069], 'l': ['57882.00000', '57868.90000'], 'h': ['61207.00000', '61207.00000'], 'o': '58110.00000'}}


In [ ]:
def get_ethprice():
    
    Ethereum_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"ETH"+"USD"
    response = requests.get(Ethereum_api_url)
    response_json = response.json()
    price_USDxETH = response_json["result"]['XETHZUSD']['a'][0]
    return price_USDxETH
#ethereum, bitcoin, ripple, chainlink, ada, litecoin, filecoin, dot
def get_rippleprice():
    
    Ripple_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"XRP"+"USD"
    response = requests.get(Ripple_api_url)
    response_json = response.json()
    price_USDxXRP = response_json[]

In [4]:
import requests
Ripple_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"XRP"+"USD"
response = requests.get(Ripple_api_url)
response_json = response.json()
#price_USDxXRP = esponse_json[]
print(response_json)

{'error': [], 'result': {'XXRPZUSD': {'a': ['1.44205000', '149', '149.000'], 'b': ['1.44013000', '149', '149.000'], 'c': ['1.44014000', '180.40738738'], 'v': ['4764061.19961517', '96047002.72505535'], 'p': ['1.40964040', '1.24462751'], 't': [2282, 49782], 'l': ['1.34774000', '1.01803000'], 'h': ['1.44999000', '1.44999000'], 'o': '1.37056000'}}}


In [ ]:
just work on the fetch and stake of the code
stake in the amount of shares that a person has in percentages

In [ ]:
def fetch_request(intent_request):
    '''
    Code performs fetching of the price of the bitcoin
    
    '''
    #Import the requests library
import requests
TICKER_API_URL = 'https://api.coinmarketcap.com/v1/ticker/'
def get_latest_crypto_price(crypto):
  
  response = requests.get(TICKER_API_URL+crypto)
  response_json = response.json()
  
  return float(response_json[0]['price_usd'])
get_latest_crypto_price('bitcoin')

In [16]:
import requests
#https://api.kraken.com/0/public/Ticker?pair=XBTUSD
#ETH = ethereum, DOT = polkadot, FIl = filecoin, LINK = chainlink,
#LTC = litecoin, cardano = ADA, ripple = XEP, bitcoin = XBT
#
crypto =["ETH", "DOT", "FIL", "LINK", "LTC", "ADA", "XRP", "XBT"]
coin_api_url = "https://api.kraken.com/0/public/Ticker?pair=" + str(crypto) + "USD" 

#Ripple_api_url = "https://api.kraken.com/0/public/Ticker?pair="+"XRP"+"USD"
def fetch_request(intent_request):
    
    '''
    Code performs fetching of the price of the bitcoin 
    '''
    response = requests.get(coin_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd
    
    return float(response_json[0]['price_usd'])

In [19]:
crypto =["ETH", "DOT", "FIL", "LINK", "LTC", "ADA", "XRP", "XBT"]
coin_api_url = "https://api.kraken.com/0/public/Ticker?pair=" + str(crypto) + "USD" 

def get_coin_price(crypto):
    """
    Retrieves the current price of bitcoin in US Dollars from the kraken.com Crypto API.
    """
    
    #bitcoin_api_url = "https://api.kraken.com/0/public/Ticker?pair="+coin+"USD"
    response = requests.get(coin_api_url)
    response_json = response.json()
    price_usd = parse_float(response_json["data"]["1"]["quotes"]["USD"]["price"])
    return price_usd

get_coin_price(crypto[0])

price_USDxETH = response_json["result"]['XETHZUSD']['a'][0]

KeyError: 'data'